
Задание 1.
Обязательная часть

Будем парсить страницу со свежими новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>



In [1]:
KEYWORDS = ['design', 'java']

In [2]:
import requests
import json

In [3]:
from bs4 import BeautifulSoup

In [4]:
import time
from datetime import datetime

In [5]:
import pandas as pd

In [6]:
req = requests.get('https://habr.com/ru/all/')
time.sleep(0.3)
soup = BeautifulSoup(req.text, 'html.parser')

In [7]:
posts = soup.find_all('article', class_='post')


In [8]:
habr_news = pd.DataFrame()
findings_list = []
for p in posts:
    findings = p.find('div', class_='post__text')
    for k in KEYWORDS:
        if k in findings.text.lower():
            date = p.find('span', class_='post__time').text
            title = p.find('h2', class_='post__title').text.strip()
            link = p.find('a', class_='post__title_link').get('href')
            article = BeautifulSoup(requests.get(link).text, 'html.parser')
            time.sleep(0.3)
            text = article.find('div', class_='post__text').text.strip()
            row = {'date': date, 'title': title, 'link': link, 'text': text}
            habr_news = pd.concat([habr_news, pd.DataFrame([row])])
            
habr_news

,date,title,link,text
0,сегодня в 11:21,Fuchsia OS: Возможности и перспективы развития,https://habr.com/ru/post/518068/,"Хабр, привет! В данной статье предлагаю рассмо..."
0,сегодня в 11:14,Обработка ошибок в JavaScript,https://habr.com/ru/post/518050/,"Привет, Хабр!Тема обработки ошибок в JavaScrip..."
0,сегодня в 10:23,Передовой опыт тестирования в Java,https://habr.com/ru/company/funcorp/blog/517724/,"Чтобы покрытие кода было достаточным, а создан..."



Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.


In [9]:
EMAIL = ['vasya@ya.ru', 'vasya@gmail.com']

In [10]:
def check_breaches(emails):
    breaches = pd.DataFrame()
    URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
    headers = {'Vaar-Version': '0', 'Vaar-Header-App-Product': 'hackcheck-web-avast'}
    for e in EMAIL:
        req = requests.post(URL, headers=headers,json={"emailAddresses":[e]})
        time.sleep(0.3)
        for v in req.json()['breaches'].values():
            breaches_dict = {'e-mail': e, 'date': v['publishDate'], 'source': v['site'], 'description': v['description']}
            breaches = pd.concat([breaches, pd.DataFrame([breaches_dict])])
    return breaches

In [11]:
check_breaches(EMAIL)

,e-mail,date,source,description
0,vasya@ya.ru,2016-10-26T00:00:00Z,twitter.com,Login credentials for over 32 Million Twitter ...
0,vasya@ya.ru,2019-03-28T00:00:00Z,verifications.io,Big data e-mail verification platform verifica...
0,vasya@ya.ru,2020-05-21T00:00:00Z,vk.com,"At some time in 2020, the Russian social netwo..."
0,vasya@ya.ru,2016-11-01T00:00:00Z,qip.ru,"In 2011, Russian instant messaging service pro..."
0,vasya@ya.ru,2018-06-19T00:00:00Z,storelp.ru,"At an unconfirmed date, StoreLP.ru's database ..."
...,...,...,...,...
0,vasya@gmail.com,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
0,vasya@gmail.com,2018-02-03T00:00:00Z,lsgb.net,"On an unconfirmed date, the manufacturing comp..."
0,vasya@gmail.com,2017-03-31T00:00:00Z,fc-zenit.ru,"In July 2010, FC Zenit's user database was all..."
0,vasya@gmail.com,2018-07-10T00:00:00Z,lpfiction.com,"At an unconfirmed date, LPfiction's database w..."



Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.getGROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.


In [12]:
def get_group_posts(group):
    WALL_REQUEST = 'https://api.vk.com/method/wall.get?'
    TOKEN = '...'
    VERSION = '5.21'
    SLEEP = 0.33
    params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': group,
    'count': 50
    }
    vk_group = requests.get(WALL_REQUEST, params)
    data = pd.DataFrame(vk_group.json()['response']['items'])[['date', 'text']]
    data['date'] =  data['date'].map(lambda x: datetime.fromtimestamp(x))
    return data

In [13]:
get_group_posts('netology')

,date,text
0,2020-09-07 11:00:10,Мы снова дарим подарки 🎁\n \nНа этот раз всё с...
1,2020-09-06 17:12:00,"И так бывает: постоянно занят, а дел и задач с..."
2,2020-09-05 16:48:00,В новой афише мероприятий: круглый стол по воп...
3,2020-09-05 11:12:00,"Мы проводим на работе большую часть жизни, и х..."
4,2020-09-04 18:42:00,"Учебный год официально начался, начинаем учить..."
5,2020-09-04 10:50:00,Вот раньше были времена: реклама не гонялась з...
6,2020-09-03 18:42:00,"Голос дрожит, ладошки потеют, все слова вылете..."
7,2020-09-03 10:50:00,Вот бы существовала волшебная пилюля для бизне...
8,2020-09-02 18:12:00,"Есть бизнес-идея, свой стартап или небольшой о..."
9,2020-09-02 10:24:00,"Дизайнеров, кажется, не остановить🤺 \nПрисказк..."
